In [1]:
import pandas as pd


## Energy Consumption for heating in EU

Data from:
https://energy.ec.europa.eu/topics/energy-efficiency/energy-efficient-buildings/eu-building-stock-observatory_en

Also has data on age of buildings, not easily used though


In [24]:

consumption = pd.DataFrame(pd.read_excel("./energy_consumption.xlsx"))
columns = consumption.iloc[3].dropna()[1:-2].tolist()
consumption = consumption.iloc[4567:4771]
columns = ["Category", "Country", "Unit"] + [str(i) for i in columns]
columns = [i if "." not in i else str(int(float(i))) for i in columns]

blocks = consumption.iloc[:,6]
block_starts = blocks[blocks == "EU"].index.tolist()

heating_consumption = []

for i, start in enumerate(block_starts[:-1]):
    block_data = consumption.loc[start:block_starts[i+1]]
    category = block_data.iloc[:,4].dropna()
    data = block_data.iloc[:,6:-2]
    data.columns = columns[1:]
    data["Category"] = category.values[0]
    data = data[data.columns]

    heating_consumption.append(data)

heating_consumption = pd.concat(heating_consumption)


In [25]:
# Checking that the data makes sense

total = heating_consumption[heating_consumption["Category"] == "Residential-Space heating"]
f_total = total[total["Country"] == "Finland"]
other = heating_consumption[heating_consumption["Category"] != "Residential-Space heating"]
f_other = other[other["Country"] == "Finland"]

print(f_total["2000"].values[0], f_other["2000"].sum())


2.8092 2.8092


In [47]:
#Figures from the model provided by Avoin, those categories that match, added that
#renewables have a co2-emission of 0, which is disputable:

emission = {'Natural gas':0.199, 'Petroleum products':0.255, 'Solid fuel':0.354, 'Electricity':0.104, 'Derivated heat':0.195, 'Renewables':0.104}

In [48]:
year = "2017"
import math
name = []
co2_kwh = []
for country in set(heating_consumption["Country"]):
    print(f"---------- {country} ----------")
    country_data = heating_consumption[heating_consumption["Country"] == country]
    country_total = country_data[country_data["Category"] == "Residential-Space heating"][year].values[0]
    co2_per_unit = 0
    for source in country_data["Category"]:
        if source != "Residential-Space heating":
            from_source = country_data[country_data["Category"] == source][year].values[0]
            if not math.isnan(from_source):
                co2_per_unit += from_source/country_total * emission[source]
            print(f"Fraction produced by {source}: {from_source / country_total}")
    name.append(country)
    co2_kwh.append(co2_per_unit)


---------- France ----------
Fraction produced by Natural gas: 0.3678995508588539
Fraction produced by Petroleum products: 0.14973657243210503
Fraction produced by Solid fuel: 0.008502824751143806
Fraction produced by Electricity: 0.11307766446604013
Fraction produced by Derivated heat: 0.04529125603331035
Fraction produced by Renewables: 0.31548451243995257
---------- Slovakia ----------
Fraction produced by Natural gas: 0.7060087483692733
Fraction produced by Petroleum products: 0.0053718056941140365
Fraction produced by Solid fuel: 0.021257002532422686
Fraction produced by Electricity: 0.028624050341493363
Fraction produced by Derivated heat: 0.21379786662573866
Fraction produced by Renewables: 0.024940526436958026
---------- Romania ----------
Fraction produced by Natural gas: 0.17285598512656716
Fraction produced by Petroleum products: 0.0
Fraction produced by Solid fuel: 0.0013347952519426037
Fraction produced by Electricity: 0.012108499785479333
Fraction produced by Derivated he

In [49]:
co2rates = pd.DataFrame(index=name, data={'co2/kWh':co2_kwh})
co2rates

,co2/kWh
France,0.167807
Slovakia,0.196652
Romania,0.140274
Czech Republic,0.181818
Bulgaria,0.153273
Portugal,0.115658
Croatia,0.138947
Spain,0.174502
Slovenia,0.146709
Estonia,0.153030


In [50]:
co2rates.to_csv('co2rates.csv')